#Baixando o Spark

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

!rm -rf spark-2.4.4-bin-hadoop2.7.tgz

#Baixando o java 8

In [ ]:
!apt update 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

#Importando variáveis de ambiente

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

#Importando funções

In [ ]:
def limpeza(text): #Criando estrutura de dados para o arquivo da amazon
  text = text.split('\r\n')
  flag = 0
  vet = {}
  for index,words in enumerate(text):

    if words.startswith('Id:'):
      vet['id'] = words.split()[1].strip()

    elif words.startswith('ASIN:'):
      vet['asin'] = words.split('ASIN:')[1].strip()
        
    elif words.startswith('  title:'):
      vet['title'] = words.split('title: ')[1].strip()

    elif words.startswith('  group:'):
      vet['group'] = words.split('group:')[1].strip()

    elif words.startswith('  salesrank:'):
      vet['salesrank'] = words.split('salesrank:')[1].strip()

    elif words.startswith('  similar:'):
      if words.strip()[9] != 0:
        vet['similar'] = words.split()[2:]
      
    elif words.startswith('  categories:'):
      vet['categories'] = []

    elif words.startswith('   |'):
      categories = words.split('|')
      if len(categories) == 1:
          pass
      else:
          vet['categories'].append(categories[len(categories)-1])

    elif words.startswith('  reviews:'):
      vet['avg_rating'] =  words.split(': ')[4]
      vet['reviews'] = []

    elif words.startswith('    2') or words.startswith('    1'):
      aux = words.split()
      vet['reviews'].append({'data':aux[0], 'cliente':aux[2], 
                             'rating':aux[4], 'votes':aux[6], 'helpful':aux[8]})
            
  return vet

def limpeza2(word): #remove o id[0]
  if len(word) != 0 and len(word) > 2 :
    return word

def transformList(word): #transformar para lista para fazer melhor manipulações
  lista = []
  for i in word:
    for j in i:
      lista.append(j)

  return lista

def media(anterior, valor, tamanho): #media de diaas
  vet = float(anterior) + (float(valor)-float(anterior))/float(tamanho)
  return round(vet,5)

def oi2(busca): #Seleciona dados com base group == group e media in lista_media[]
  global filtrado
  contador = 0
  vet = {}
  vet['result'] = []
  for i in filtrado:
    if i[0] == busca[1] and busca[2] in i[1]:
      vet['result'].append({'title': busca[0], 'group': busca[1], 'avg':busca[2]}) 
      contador+=1

  return vet



#Importanto biblioteca e Escolhendo o Produto


In [ ]:
from pyspark import SparkConf, SparkContext, SparkFiles, SQLContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

import operator
import re

ID_PRODUTO = '2'

conf = SparkConf().setMaster("local[*]").setAppName("Spark")
sc = SparkContext.getOrCreate(conf=conf)

sc._jsc.hadoopConfiguration().set("textinputformat.record.delimiter", "\r\n\r\n")

#Colocando o arquivo amazon-meta.txt no RDD

In [ ]:
url = "https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz"

sc.addFile(url)
file_name = re.split(pattern='/', string=url)[-1]
file_name = SparkFiles.get(file_name)
text_file = sc.textFile(file_name)

#Testando o arquivo no RDD

In [ ]:
print(text_file.take(2))

['# Full information about Amazon Share the Love products\r\nTotal items: 548552', 'Id:   0\r\nASIN: 0771044445\r\n  discontinued product']


In [ ]:
import pyspark
spark = pyspark.sql.SparkSession.builder \
            .master('local') \
            .appName('Spark') \
            .getOrCreate()

#1) Dado produto, listar os 5 comentários mais úteis e com maior avaliação e os 5 comentários mais úteis e com menor avaliação


In [ ]:
def produtos(word):
  return (int(word['id']), word['asin'], word['title'], word['group'], int(word['salesrank']), float(word['avg_rating']))

def similares(word):
  vet = []
  for i in word['similar']:
    vet.append((int(word['id']), i))
  
  return vet

def categorias(word):
  vet = []
  for i in word['categories']:
    vet.append((int(word['id']), i))
  
  return vet

def reviews(word):
  vet = []
  for i in word['reviews']:
    vet.append((int(word['id']), i['data'], i['cliente'], float(i['rating']), int(i['votes']), int(i['helpful'])))
  
  return vet

lines = text_file.map(limpeza).filter(limpeza2)

produtos = lines.map(produtos)
similares = lines.map(similares).flatMap(lambda x: x)
categorias = lines.map(categorias).flatMap(lambda x: x)
reviews = lines.map(reviews).flatMap(lambda x: x)


produtosDF = produtos.toDF(['id_produto', 'asin', 'title', 'group', 'salesrank', 'avg_rating'])
similaresDF = similares.toDF(['id_produto', 'similar'])
categoriasDF = categorias.toDF(['id_produto', 'categoria'])
reviewsDF = reviews.toDF(['id_produto', 'data', 'cliente', 'rating', 'votes', 'helpful'])


produtosDF.createOrReplaceTempView("produtos")
similaresDF.createOrReplaceTempView("similares")
categoriasDF.createOrReplaceTempView("categorias")
reviewsDF.createOrReplaceTempView("reviews")




#1) Dado produto, listar os 5 comentários mais úteis e com maior avaliação e os 5 comentários mais úteis e com menor avaliação


In [ ]:
id_produto = 2

comentarios_mais_uteis = spark.sql(
"""(SELECT * FROM reviews where id_produto = {}
ORDER BY helpful DESC LIMIT 5) ORDER BY rating DESC""".format(id_produto))

comentarios_mais_uteis.show()

comentarios_menos_uteis = spark.sql(
"""(SELECT * FROM reviews where id_produto = {} ORDER BY helpful 
DESC LIMIT 5) ORDER BY rating ASC""".format(id_produto))

comentarios_menos_uteis.show()

+----------+---------+--------------+------+-----+-------+
|id_produto|     data|       cliente|rating|votes|helpful|
+----------+---------+--------------+------+-----+-------+
|         2|2002-1-24|A13SG9ACZ9O5IM|   5.0|    8|      8|
|         2|2002-5-23|A1GIL64QK68WKL|   5.0|    8|      8|
|         2| 2002-2-6|A2P6KAWXJ16234|   4.0|   16|     16|
|         2|2002-3-23|A3GO7UV9XX14D8|   4.0|    6|      6|
|         2|2004-2-11|A1CP26N8RHYVVO|   1.0|   13|      9|
+----------+---------+--------------+------+-----+-------+

+----------+---------+--------------+------+-----+-------+
|id_produto|     data|       cliente|rating|votes|helpful|
+----------+---------+--------------+------+-----+-------+
|         2|2004-2-11|A1CP26N8RHYVVO|   1.0|   13|      9|
|         2|2002-3-23|A3GO7UV9XX14D8|   4.0|    6|      6|
|         2| 2002-2-6|A2P6KAWXJ16234|   4.0|   16|     16|
|         2|2002-1-24|A13SG9ACZ9O5IM|   5.0|    8|      8|
|         2|2002-5-23|A1GIL64QK68WKL|   5.0|    8|     

#2) Dado um produto, listar os produtos similares com maiores vendas do que ele


In [ ]:
id_produto = 2

produtos_similares = spark.sql("""SELECT * FROM produtos as p INNER JOIN 
similares as s ON p.asin = s.similar WHERE s.id_produto = {0} and p.salesrank > 
(SELECT salesrank FROM produtos WHERE id_produto = {0})""".format(id_produto)) 

produtos_similares.show()

+----------+----------+--------------------+-----+---------+----------+----------+----------+
|id_produto|      asin|               title|group|salesrank|avg_rating|id_produto|   similar|
+----------+----------+--------------------+-----+---------+----------+----------+----------+
|    411179|1567182836|Beltane: Springti...| Book|   280541|       3.5|         2|1567182836|
|    454888|0738700827|Ostara: Customs, ...| Book|   311254|       4.0|         2|0738700827|
+----------+----------+--------------------+-----+---------+----------+----------+----------+



#3) Dado um produto, mostrar a evolução diária das médias de avaliação ao longo do intervalo de tempo coberto no arquivo de entrada

In [ ]:
id_produto = 2

evolucao = spark.sql(""" SELECT id_produto, data, rating, (SUM(rating) OVER 
(ORDER BY data))/(ROW_NUMBER()  OVER (ORDER BY data)) as average FROM reviews
WHERE id_produto = {} ORDER BY data""".format(id_produto)) 

evolucao.show()




+----------+----------+------+------------------+
|id_produto|      data|rating|           average|
+----------+----------+------+------------------+
|         2|2001-12-16|   5.0|               5.0|
|         2| 2002-1-24|   5.0|               5.0|
|         2| 2002-1-28|   5.0|               5.0|
|         2|  2002-1-7|   4.0|              4.75|
|         2| 2002-2-14|   4.0|               4.6|
|         2|  2002-2-6|   4.0|               4.5|
|         2| 2002-3-23|   4.0| 4.428571428571429|
|         2| 2002-5-23|   5.0|               4.5|
|         2|2003-11-25|   5.0| 4.555555555555555|
|         2| 2003-2-25|   5.0|               4.6|
|         2| 2004-2-11|   1.0|4.2727272727272725|
|         2|  2005-2-7|   5.0| 4.333333333333333|
+----------+----------+------+------------------+



#4) Listar os 10 produtos líderes de venda em categoria

In [ ]:
categoria = "Criminology[11005]"

lideres_categorias = spark.sql(""" 
SELECT p.id_produto, p.title, c.categoria, p.salesrank FROM categorias as c
INNER JOIN produtos as p ON c.id_produto = p.id_produto WHERE c.categoria = '{}' 
ORDER BY p.salesrank DESC LIMIT 10""".format(categoria))

lideres_categorias.show()



+----------+--------------------+------------------+---------+
|id_produto|               title|         categoria|salesrank|
+----------+--------------------+------------------+---------+
|    196343|Uniform Crime Rep...|Criminology[11005]|  3158228|
|     30755|Searching for a D...|Criminology[11005]|  3092830|
|    431227|Violent Racism: V...|Criminology[11005]|  3049934|
|    223783|     Corporate Fraud|Criminology[11005]|  2662778|
|     78671|European Democrac...|Criminology[11005]|  2651679|
|     25715|Money Laundering ...|Criminology[11005]|  2422943|
|     18797|Tradition of the ...|Criminology[11005]|  2411074|
|     84905|Common Sense Poli...|Criminology[11005]|  2385686|
|    514491|Black-Jewish Rela...|Criminology[11005]|  2297994|
|    307584|European Democrac...|Criminology[11005]|  2248960|
+----------+--------------------+------------------+---------+



#5) Listar os 10 produtos com a maior média de ratings por grupo (Group) de produtos

In [ ]:
produtos_grupos = spark.sql(""" 
SELECT id_produto, title, group, avg_rating FROM (SELECT *, ROW_NUMBER() OVER 
(PARTITION BY group ORDER BY avg_rating DESC) AS row FROM produtos) 
WHERE row <= 10 
""") 

produtos_grupos.show(1000)


+----------+--------------------+------------+----------+
|id_produto|               title|       group|avg_rating|
+----------+--------------------+------------+----------+
|        71|Jonny Quest - Ban...|       Video|       5.0|
|       180|Barney: Sing and ...|       Video|       5.0|
|       229|Bob Newhart - But...|       Video|       5.0|
|       251| The Glass Menagerie|       Video|       5.0|
|       391|National Geograph...|       Video|       5.0|
|       415|     Wow Gospel 1999|       Video|       5.0|
|       442|It's Arbor Day, C...|       Video|       5.0|
|       669|   The Sweetest Gift|       Video|       5.0|
|       873|The Best of The S...|       Video|       5.0|
|       887|Dixie Carter's Un...|       Video|       5.0|
|       922|Party Tyme Karaok...|         Toy|       5.0|
|    257106|  IlluStory Book Kit|         Toy|       5.0|
|    305664|Photostory Junior...|         Toy|       5.0|
|    465900|Wizard Card Game ...|         Toy|       5.0|
|    327405|Pa

#6) Listar a 5 categorias de produto com a maior média geral de ratings 

In [ ]:
categoria_media = spark.sql(""" 
SELECT categoria, AVG(avg_rating) FROM produtos as p INNER JOIN categorias as c 
ON p.id_produto = c.id_produto GROUP BY categoria ORDER BY avg(avg_rating) DESC 
LIMIT 5 
""") 

categoria_media.show()

+--------------------+---------------+
|           categoria|avg(avg_rating)|
+--------------------+---------------+
|Johnson, Brooke[1...|            5.0|
|CDs Under $7[519436]|            5.0|
|     Political[5099]|            5.0|
|  Craft Kits[288170]|            5.0|
|Singer, Isaac Bas...|            5.0|
+--------------------+---------------+



#7) Listar os 10 clientes que mais fizeram comentários por grupo (Group) de produto

In [ ]:
group = "Book"

comentarios_grupo = spark.sql(""" 
SELECT group,cliente,count(cliente) as quantidade_de_comentarios FROM reviews INNER JOIN produtos 
ON reviews.id_produto = produtos.id_produto  WHERE group = '{}' GROUP BY group, 
cliente ORDER BY group, quantidade_de_comentarios DESC LIMIT 10
""".format(group))

comentarios_grupo.show(50)


+-----+--------------+-------------------------+
|group|       cliente|quantidade_de_comentarios|
+-----+--------------+-------------------------+
| Book| ATVPDKIKX0DER|                   643185|
| Book|A3UN6WX5RRO2AG|                   154531|
| Book|A14OJS0VWMOSWO|                     9589|
| Book|   AFVQZQ8PW0L|                     5441|
| Book|A1K1JW1C5CUSUZ|                     3562|
| Book|A2NJO6YE954DBH|                     2055|
| Book|A3QVAKVRAH657N|                     1651|
| Book|A1NATT3PN24QWY|                     1535|
| Book|A1D2C0WDCSHUWZ|                     1508|
| Book|A2ODBHT4URXVXQ|                     1469|
+-----+--------------+-------------------------+

